In [1]:
from pprint import pprint

import folium
from bson import SON
from dotenv import load_dotenv
import os 
import logger_setup
import logging
from pymongo import MongoClient, GEO2D
from pymongo.database import Database, Collection
from csv_reader import ais_csv_to_gdf, split_df_by_batch_size

import time

import numpy as np
from dotenv import load_dotenv

import logger_setup

import pandas as pd
from influxdb_client import InfluxDBClient
# Ciekawy statek:
# 215131000

import os
from influxdb_client import Point
from influxdb_client.client.write_api import SYNCHRONOUS

from csv_reader import ais_csv_to_df

import psycopg2

logger_setup.setup_logging(level=logging.INFO)
logger = logging.getLogger()
load_dotenv()

True

---

# Setup

## Mongo setup

In [3]:
mongo_url = "mongodb://localhost:" + os.environ.get("MONGO_PORT", "55000")
mongo_database = "temp"
mongo_collection = "temp"

logger.info(f"MongoDB endpoint: {mongo_url}")
logger.info(f"MongoDB database name: {mongo_database}")
logger.info(f"MongoDB collection name: {mongo_collection}")

2024-08-05 18:07:41,116 - INFO: MongoDB endpoint: mongodb://localhost:55000
2024-08-05 18:07:41,117 - INFO: MongoDB database name: temp
2024-08-05 18:07:41,118 - INFO: MongoDB collection name: temp


## Influx setup

In [ ]:

influx_token = os.environ.get("API_INFLUX_KEY")
influx_org = os.environ.get("INFLUX_ORG_ID")
influx_url = "http://localhost:" + os.environ.get("INFLUX_PORT", "55000")

logger.debug(f"InfluxDB Token: {influx_token}")
logger.debug(f"InfluxDB Organization id: {influx_org}")
logger.info(f"InfluxDB  Database endpoint: {influx_url}")

## MobilityDB setup

In [ ]:
mobility_host       = os.environ.get("MOBILITY_HOST")
mobility_port       = os.environ.get("MOBILITY_PORT")
mobility_user       = os.environ.get("MOBILITY_USER")
mobility_password   = os.environ.get("MOBILITY_PASSWORD")
mobility_database   = os.environ.get("MOBILITY_DATABASE")

# Queries

---

## Spatial queries

---

### Datasest

In [74]:
class Bbox:
    def __init__(self, min_lon, min_lat, max_lon, max_lat):
        self.min_lon = min_lon
        self.min_lat = min_lat
        self.max_lon = max_lon
        self.max_lat = max_lat
    
    def get_coords(self):
        return [self.min_lon, self.min_lat, self.max_lon, self.max_lat]
    
bounding_boxes = [
    Bbox(-123.247925, 48.136125, -122.739476, 48.362910),  # Puget Sound, Washington
    Bbox(-123.016525, 37.639830, -122.283450, 37.929824),  # San Francisco Bay, California
    Bbox(-76.510574, 37.973348, -75.962608, 38.393338),    # Chesapeake Bay, Maryland/Virginia
    Bbox(-88.161018, 30.334953, -87.927567, 30.639975),    # Mobile Bay, Alabama
    Bbox(-95.104218, 29.327599, -94.617409, 29.623018),    # Galveston Bay, Texas
    Bbox(-82.775543, 27.599938, -82.320755, 27.934847),    # Tampa Bay, Florida
    Bbox(-122.019295, 36.776848, -121.819153, 37.018274),  # Monterey Bay, California
    Bbox(-71.484741, 41.454842, -71.173431, 41.735072),    # Narragansett Bay, Rhode Island
    Bbox(-117.253113, 32.600235, -117.085083, 32.736514),  # San Diego Bay, California
    Bbox(-88.135986, 44.474116, -87.745605, 44.794497),    # Green Bay
    
    Bbox(-80.45290918232058, 29.060643707480367, -78.32704059023007, 31.29195079716895),  # Georgia coast
    Bbox(-77.36586166597581, 31.282283517600803, -75.27345529761102, 33.760865420475),  # North Carolina coast
    Bbox(-74.51771061340146, 34.75075477385059, -71.85177110891351, 37.752840882799006),  # Delaware Bay area
    Bbox(-73.8132903076812, 39.28679980551155, -69.84951150110152, 40.36716788459955),  # Long Island Sound, New York
    Bbox(-87.53216792628709, 42.614159443972795, -86.57811342936832, 43.95974516921851),  # Lake Michigan
    Bbox(-88.8185473646937, 47.31402814776524, -85.97885758914987, 48.36369038185671),  # Lake Superior
    Bbox(-119.28398346805034, 28.592051721892147, -116.88328729269529, 32.65616790678931),  # Baja California coast
    Bbox(-115.22516221315561, 21.159217960533027, -112.50145598046407, 25.734344868162523),  # Gulf of California 0
    Bbox(-111.04125138719715, 20.598750857797, -105.33212892067695, 22.5805686602515),  # Northern Gulf of California 0
    Bbox(-96.79281541240942, 25.885529478254256, -93.32496228352338, 28.099129345180913),  # Gulf of Mexico
    Bbox(-88.54476002663944, 27.4284592236325, -84.0568785413999, 30.09273196827901),  # Gulf of Mexico, Alabama
    Bbox(-128.9871203472166, 40.000911885515904, -125.04987957500578, 48.818149529347096),  # Alaska coast
    Bbox(-126.4987002469207, 35.49916842114703, -122.76107863128735, 38.76867969769498),  # Off the coast of California
    Bbox(-90.41838995337584, 30.039892740717292, -89.89240461303508, 30.37633672889592),  # Mississippi coast
    Bbox(-123.02323990734207, 32.99070496259917, -119.56454319294964, 35.24101423046929)  # Northern California coast
]

---

### MongoDB

In [76]:
# Mongo

client: MongoClient = MongoClient(mongo_url)
db: Database = client[mongo_database]
collection: Collection = db[mongo_collection]

mongo_spatial_times = []

logger.info(f"Running Mongo spatial")
for bbox in bounding_boxes:
    min_lon, min_lat, max_lon, max_lat = bbox.get_coords()
    
    # Define the query using $geoWithin and $box
    query = {
        'geometry': {
            '$geoWithin': {
                '$box': [
                    [min_lon, min_lat],
                    [max_lon, max_lat]
                ]
            }
        }
    }
    start_time = time.time()
    # Execute the query
    results = list(collection.find(query, {'_id': False}))
    end_time = time.time()
    logger.info(f"Query time: {end_time - start_time}, no. of results: {len(results)}")
    mongo_spatial_times.append(end_time - start_time)

logger.info(f"Mongo spatial average time: {sum(mongo_spatial_times) / len(mongo_spatial_times)}")

client.close()

2024-08-05 20:27:58,361 - INFO: Running Mongo spatial
2024-08-05 20:27:58,362 - INFO: {"message": "Waiting for suitable server to become available", "selector": "Primary()", "operation": "find", "topologyDescription": "<TopologyDescription id: 66b119ae48826bcaa1202004, topology_type: Unknown, servers: [<ServerDescription ('localhost', 55000) server_type: Unknown, rtt: None>]>", "clientId": {"$oid": "66b119ae48826bcaa1202004"}, "remainingTimeMS": 29}
2024-08-05 20:28:07,054 - INFO: Query time: 8.691434621810913, no. of results: 2611
2024-08-05 20:28:15,663 - INFO: Query time: 8.60830807685852, no. of results: 94798
2024-08-05 20:28:23,391 - INFO: Query time: 7.7273173332214355, no. of results: 3813
2024-08-05 20:28:31,292 - INFO: Query time: 7.899705410003662, no. of results: 3242
2024-08-05 20:28:41,270 - INFO: Query time: 9.977306842803955, no. of results: 102114
2024-08-05 20:28:49,622 - INFO: Query time: 8.35139799118042, no. of results: 58465
2024-08-05 20:28:57,187 - INFO: Query t



---

### InfluxDB

In [ ]:
# Influx

---

### MobilityDB

In [ ]:
# Mobility

conn = psycopg2.connect(
    database    = mobility_database,
    host        = mobility_host,
    user        = mobility_user,
    password    = mobility_password,
    port        = mobility_port,
)

cursor = conn.cursor()

cursor.execute(f'''
select mmsi, st_astext((attime(route,
                     tstzspan('[2020-12-31T00:00:00Z, 2020-12-31T00:01:00Z]')
              )
    )::geometry) as route
from aggregated_vessel_positions
where eintersects(st_setsrid(ST_MakeEnvelope(-88.0, 41.80, -87.0, 41.87), 4326), route);
''')

print(cursor.fetchmany(size=10))

conn.close()

---

## Temporal queries

---

### MongoDB

In [17]:
# Mongo

---

### InfluxDB

In [18]:
# Influx

---

### MobilityDB

In [19]:
# Mobility

---

## Spatiotemporal queries

---

### MongoDB

In [20]:
# Mongo

---

### InfluxDB

In [21]:
# Influx

---

### MobilityDB

In [22]:
# Mobility

---

## Results

---

In [23]:
# Results

# Visualisations

---

In [ ]:
# mongo
import folium

def get_mongo_results_folium(results, *geojsons):
    CHICAGO_COORDINATES = (42, -95)
    
    map_attributions = ('&copy; <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> '
                        'contributors, &copy; <a href="http://cartodb.com/attributions">CartoDB</a>')
    
    m = folium.Map(location=CHICAGO_COORDINATES,
                   attr=map_attributions,
                   zoom_start=5,
                   control_scale=True,
                   height=800,
                   width=1400)
    for entry in results:
        geojson = (folium.GeoJson(entry))
        popup = folium.Popup(f"mmsi: {entry['properties']['MMSI']}, name {entry['properties']['VesselName']}, time: {entry['properties']['BaseDateTime']}")
        popup.add_to(geojson)
        geojson.add_to(m)
    
    for geojson in geojsons:
        folium.GeoJson(data=geojson).add_to(m)
        
    return m

m: folium.Map = get_mongo_results_folium(results)
m